## Create the US Credit Card Interchange Index Table

In [35]:
import glob #import the glob library to detect a certainly file type within a given path
import pandas as pd #import the pandas libraries to arrange the data

In [36]:
path =r'G:\Pricing\IC Opt project\ic_generalized_tables\us' #identify the path needed for the glob library
filenames = glob.glob(path + "/*.csv") #instruct the glob object to locate all csv repositories

In [37]:
g_tbls=[] #create a new list
for filename in filenames: #for every file in the directory add to the list
    g_tbls.append(pd.read_csv(filename))

In [38]:
#combine all of the generalized table csvs created from database data and IC published tables
g_tbls= pd.concat(g_tbls, ignore_index=True) 

In [39]:
# the IC tables have data that was validated and invalidate via a manual review, 
#we only want valide for the creation of our index table
g_tbls=g_tbls[(g_tbls['VALID_RATE']=='Yes')] 

In [40]:
#The original table has more columns, many were required just in the creation process
#for the development of an index table not all are needed and are therefore removed here
#Only those columns required have referenced below
g_tbls=g_tbls[['CARD_SCHEME'
        ,'CONS_CORP_BIN'
        ,'DEBIT_CREDIT'
        ,'IC_PERC_RT'
        ,'INTERCHANGE_FLAT_RATE'
        ,'INTERCHANGE_RATE'
        ,'LEVEL'
        ,'PROGRAM_NAME']]

## Create a Generalized Long Descriptor Column
The long generalized descriptor will be used to provide a full description of the IC. It complements the short descriptor that was create for ease of descriptor identification, joins, and interpreation on a larger scale 

In [41]:
#Create a new column named Generalized Descriptor through the combination of multiple attributes spaced by underscores
g_tbls['GENERALIZED_DESCRIPTOR']=\
g_tbls['CARD_SCHEME']+"_"+\
g_tbls['CONS_CORP_BIN']+"_"+\
g_tbls['DEBIT_CREDIT']+"_"+\
g_tbls['LEVEL']+"_"+\
g_tbls['PROGRAM_NAME']

In [42]:
# g_tbls.info() - used during script development

## Create The Program Level Numeric Reference Table

In [43]:
g_levels_working=g_tbls['LEVEL'].drop_duplicates() #remove duplicates data...
g_levels_working.sort('LEVEL') #sort the data alphabetically, this is how the data will be organized...
g_levels_working=pd.DataFrame(g_levels_working) #establish the filtered data as its own dataframe...
g_levels_working=g_levels_working.reset_index() #reset the index to allow for concat with numeric ref tbale...
g_levels_working=g_levels_working['LEVEL'] #only select the LEVEL column and remove the Index column...


C:\Users\djohnson\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: sort is deprecated, use sort_values(inplace=True) for INPLACE sorting
  from ipykernel import kernelapp as app


In [44]:
#g_levels_working.head() # provide a sample of the list - used during script development

In [45]:
g=[] #Create a numeric list as an index against the level numeric reference table
for i in range (1, len(g_levels_working)+1):
    g.append(i)

In [46]:
g=pd.DataFrame.from_dict(g)
#g.head() #provided a sample - used during script development

In [47]:
g_levels=pd.concat([g_levels_working,g],axis=1)

In [48]:
g_levels.rename(columns={0: 'LEVEL_INDEX'}, inplace=True)

In [49]:
# g_levels.head() # provide a sample of the list - used during script development

In [50]:
g_levels['LEVEL_INDEX']=g_levels['LEVEL_INDEX'].astype(str)

## Create The Program Name Numeric Reference Table

In [51]:
g_program_working=g_tbls['PROGRAM_NAME'].drop_duplicates() #remove duplicates data...
g_program_working.sort('PROGRAM_NAME') #sort the data alphabetically, this is how the data will be organized...
g_program_working=pd.DataFrame(g_program_working) #establish the filtered data as its own dataframe...
g_program_working=g_program_working.reset_index() #reset the index to allow for concat with numeric ref tbale...
g_program_working=g_program_working['PROGRAM_NAME'] #only select the LEVEL column and remove the Index column...

C:\Users\djohnson\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: sort is deprecated, use sort_values(inplace=True) for INPLACE sorting
  from ipykernel import kernelapp as app


In [52]:
# g_program_working.head() # provide a sample of the list - used during script development

In [53]:
g2=[] #Create a numeric list as an index against the program numeric reference table
for i in range (1, len(g_program_working)+1):
    g2.append(i)

In [54]:
g2=pd.DataFrame.from_dict(g2)
# g2.head() - used during script development

In [55]:
g_program=pd.concat([g_program_working,g2],axis=1)

In [56]:
g_program.rename(columns={0: 'PROGRAM_INDEX'}, inplace=True)

In [57]:
# g_program.head() # provide a sample of the list - used during script development

In [58]:
g_program['PROGRAM_INDEX']=g_program['PROGRAM_INDEX'].astype(str)

In [59]:
g_tbls1=pd.merge(g_tbls,g_levels,on=['LEVEL'],how='left')
g_tbls2=pd.merge(g_tbls1,g_program,on=['PROGRAM_NAME'],how='left')

In [60]:
g_tbls2.rename(columns={'LEVEL_INDEX_x': 'LEVEL_INDEX'}, inplace=True)

In [61]:
#g_tbls2 # provide a sample of the list - used during script development

In [62]:
#g_tbls2=g_tbls2.drop('LEVEL_INDEX_y',axis=1) #remove the second creation of the LEVEL_INDEX column from the merge program and level merges

## Create The Short Generalized Descriptor Column

In [63]:
#give the first three letters of any string data and append to the numeric references to create a comprehensive reference table
g_tbls2['GENERALIZED_DESCRIPTOR_CODE']=\
g_tbls2['CARD_SCHEME'].str[:1]+\
g_tbls2['CONS_CORP_BIN'].str[:3]+\
g_tbls2['DEBIT_CREDIT'].str[:2]+\
"-"+\
g_tbls2['LEVEL_INDEX']+\
"-"+\
g_tbls2['PROGRAM_INDEX']

In [64]:
#formulate the final output
g_tbls_final=g_tbls2[['CARD_SCHEME'
         ,'CONS_CORP_BIN'
         ,'DEBIT_CREDIT'
         ,'IC_PERC_RT'
         ,'INTERCHANGE_FLAT_RATE'
         ,'INTERCHANGE_RATE'
         ,'LEVEL'
         ,'PROGRAM_NAME'
         ,'LEVEL_INDEX'
         ,'PROGRAM_INDEX'
         ,'GENERALIZED_DESCRIPTOR'
         ,'GENERALIZED_DESCRIPTOR_CODE']]

In [65]:
#g_tbls_final # provide a sample of the list - used during script development

In [66]:
g_tbls_final.to_csv('G:\Pricing\IC Opt project\ic_generalized_tables\us\us_gen_tbl.csv')#export the data to shared folder for consumption/use